
# 🧪 Exercice 4 — Réseau convolutionnel sur CIFAR-10

Nous allons implémenter un réseau de type **AlexNet simplifié** pour la base **CIFAR-10** (images RGB de taille 32×32×3, 10 classes).

## 🧱 Architecture du réseau

| Couche    | Détails           | Stride | Padding | Sortie (H×W×C) | Paramètres                           |
| --------- | ----------------- | ------ | ------- | -------------- | ------------------------------------ |
| **Input** | —                 | —      | —       | 32×32×3        | —                                    |
| **Conv1** | 32 filtres 5×5    | 1      | 2       | 32×32×32       | (5×5×3×32) + 32 = **2,432**          |
| **Pool1** | MaxPool 2×2       | 2      | 0       | 16×16×32       | 0                                    |
| **Conv2** | 64 filtres 5×5    | 1      | 2       | 16×16×64       | (5×5×32×64) + 64 = **51,264**        |
| **Pool2** | MaxPool 2×2       | 2      | 0       | 8×8×64         | 0                                    |
| **Conv3** | 64 filtres 5×5    | 1      | 2       | 8×8×64         | (5×5×64×64) + 64 = **102,464**       |
| **Pool3** | MaxPool 2×2       | 2      | 0       | 4×4×64         | 0                                    |
| **FC4**   | Dense 1024 → 1000 | —      | —       | 1000           | (4×4×64)×1000 + 1000 = **1,025,000** |
| **FC5**   | Dense 1000 → 10   | —      | —       | 10             | (1000×10) + 10 = **10,010**          |

---

## 🔢 Détails techniques

### 📏 Choix des hyperparamètres

* **Convolutions** :

  * **Padding = 2**, pour garder la même taille (32×32 → 32×32)
  * **Stride = 1**, pour un balayage pixel par pixel
* **Max Poolings** :

  * **Padding = 0**, pas de padding
  * **Stride = 2**, pour réduire de moitié la taille spatiale

### 📐 Tailles de sorties couche par couche

| Couche | Taille   |
| ------ | -------- |
| Conv1  | 32×32×32 |
| Pool1  | 16×16×32 |
| Conv2  | 16×16×64 |
| Pool2  | 8×8×64   |
| Conv3  | 8×8×64   |
| Pool3  | 4×4×64   |
| FC4    | 1000     |
| FC5    | 10       |

---

## 🧮 Total des paramètres

| Couche    | Paramètres    |
| --------- | ------------- |
| Conv1     | 2,432         |
| Conv2     | 51,264        |
| Conv3     | 102,464       |
| FC4       | 1,025,000     |
| FC5       | 10,010        |
| **Total** | **1,191,170** |

---

## 📊 Comparaison avec le nombre d’exemples

* **Nombre d’exemples en entraînement** : 50,000
* **Nombre de paramètres à apprendre** : 1,191,170

→ Environ **24 paramètres par exemple**, ce qui est **raisonnable**, mais **risque de surapprentissage sans régularisation** (dropout, data augmentation, etc.).

---


## 🧪 5. Implémentation TensorFlow/Keras

In [2]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation
from tensorflow.keras.utils import plot_model

model = Sequential()

# conv1 : 32 filtres 5x5, padding='same' pour garder la taille 32x32
model.add(Conv2D(32, kernel_size=5, padding='same', activation='relu', input_shape=(32, 32, 3)))

# pool1 : division spatiale par 2
model.add(MaxPooling2D(pool_size=2, strides=2))

# conv2 : 64 filtres 5x5, même padding
model.add(Conv2D(64, kernel_size=5, padding='same', activation='relu'))
# pool2
model.add(MaxPooling2D(pool_size=2, strides=2))

# conv3 : 64 filtres
model.add(Conv2D(64, kernel_size=5, padding='same', activation='relu'))
# pool3
model.add(MaxPooling2D(pool_size=2, strides=2))

# flatten puis fully connected
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(10, activation='softmax'))

# Affichage résumé du modèle
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 32, 32, 32)     │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 16, 16, 64)     │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 8, 8, 64)       │       102,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 4, 4, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1000)           │     1,025,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │        10,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,191,170 (4.54 MB)

 Trainable params: 1,191,170 (4.54 MB)

 Non-trainable params: 0 (0.00 B)


## 🧠 En résumé

* ✅ On a respecté les tailles de **padding** et **stride** pour garantir la cohérence.
* 🎯 Le modèle est bien **adapté aux images CIFAR-10** (32×32×3).
* 🧱 Les couches **fully-connected** représentent la majeure partie des **1.2M de paramètres**.
* 🔁 Le réseau est **simple mais puissant**, à condition de bien **régulariser**.

---